In [1]:
import numpy as np
import pandas as pd
import tqdm
import collections 
import time
import pickle

In [2]:
train = pd.read_csv('data/raw182_Training_Relabeled_Auto_25.csv')
test = pd.read_csv('data/raw91_Testing_Relabeled_Auto_25.csv')

### data preprocessing

- sliding window = 40
- if there is equal or more than 25 fall -> lable of chunk is fall

In [3]:
n = 40
X_train_adl, X_train_fall, y_train_adl, y_train_fall = list(), list(), list(), list()
for i in tqdm.tqdm(range(0, len(train) - 40)):
    temp = list()
    
    temp.append(train[' ms_accelerometer_x'][i : i + n])
    temp.append(train[' ms_accelerometer_y'][i : i + n])
    temp.append(train[' ms_accelerometer_z'][i : i + n])
    
    cnt = collections.Counter(train['outcome'][i : i + n])
    num_adl = cnt[0]
    num_fall = cnt[1]
    
    if num_fall >= 25:
        y_train_fall.append(1)
        X_train_fall.append(temp)
    else:
        y_train_adl.append(0)
        X_train_adl.append(temp)

100%|██████████████████████████████████| 33980/33980 [00:01<00:00, 20333.56it/s]


In [4]:
n = 40
X_test_adl, X_test_fall, y_test_adl, y_test_fall = list(), list(), list(), list()
for i in tqdm.tqdm(range(0, len(test) - 40)):
    temp = list()
    
    temp.append(test[' ms_accelerometer_x'][i : i + n])
    temp.append(test[' ms_accelerometer_y'][i : i + n])
    temp.append(test[' ms_accelerometer_z'][i : i + n])
    
    cnt = collections.Counter(test['outcome'][i : i + n])
    num_adl = cnt[0]
    num_fall = cnt[1]
    
    if num_fall >= 25:
        y_test_fall.append(1)
        X_test_fall.append(temp)
    else:
        y_test_adl.append(0)
        X_test_adl.append(temp)

100%|██████████████████████████████████| 17189/17189 [00:00<00:00, 19367.45it/s]


In [5]:
len_train_notfall = len(X_train_adl)
len_train_fall = len(X_train_fall)
print('number of train data fall', len_train_fall)
print('number of train data not fall', len_train_notfall)

number of train data fall 2912
number of train data not fall 31068


In [6]:
len_test_notfall = len(X_test_adl)
len_test_fall = len(X_test_fall)
print('number of test data fall', len_test_fall)
print('number of test data not fall', len_test_notfall)

number of test data fall 1456
number of test data not fall 15733


In [7]:
# downsampling X train

X_train_notfall = X_train_adl[:len_train_fall]
y_train_notfall = y_train_adl[:len_train_fall]

y_train_gen = y_train_notfall + y_train_fall
X_train_gen = X_train_notfall + X_train_fall

print(len(X_train_gen))
print(len(y_train_gen))

5824
5824


In [8]:
# downsampling X test

X_test_notfall = X_test_adl[:len_test_fall]
y_test_notfall = y_test_adl[:len_test_fall]

y_test_gen = y_test_notfall + y_test_fall
X_test_gen = X_test_notfall + X_test_fall

print(len(X_test_gen))
print(len(y_test_gen))

2912
2912


In [9]:
dic_train = {}
for i in range(0,len(X_train_gen)):
    # X
    df_chunk = pd.DataFrame(X_train_gen[i]).T
    # y
    label_chunk = y_train_gen[i]
    dic_train[i] = (df_chunk,label_chunk)

In [10]:
len(dic_train)

5824

In [11]:
dic_test = {}
for i in range(0,len(X_test_gen)):
    # X
    df_chunk = pd.DataFrame(X_test_gen[i]).T
    # y
    label_chunk = y_test_gen[i]
    dic_test[i] = (df_chunk,label_chunk)

In [12]:
len(dic_test)

2912

In [13]:
with open('data/data_train.pkl', 'wb') as file:
    pickle.dump(dic_train, file)

In [14]:
with open('data/data_test.pkl', 'wb') as file:
    pickle.dump(dic_test, file)

In [15]:
# load train data
with open('data/data_train.pkl', 'rb') as file:
    data_train = pickle.load(file)

print(data_train[0])

(    ms_accelerometer_x  ms_accelerometer_y  ms_accelerometer_z
0             0.963379            0.063477           -0.036865
1             0.990723            0.057617            0.005371
2             0.990967            0.108398            0.034668
3             0.976807            0.184570            0.060791
4             0.999268            0.143311            0.055176
5             1.026367            0.109375            0.048096
6             1.011963            0.097900            0.048340
7             0.999023            0.061035            0.077148
8             0.999023            0.061035            0.077148
9             0.930176           -0.141357            0.406250
10            0.876709           -0.259766            0.372314
11            0.971680           -0.078125            0.221924
12            1.051025            0.028076            0.108887
13            1.051025            0.028076            0.108887
14            1.017334            0.125977            

In [16]:
# load test data
with open('data/data_test.pkl', 'rb') as file:
    data_test = pickle.load(file)

print(data_test[0])

(    ms_accelerometer_x  ms_accelerometer_y  ms_accelerometer_z
0             1.058105            0.096436            0.113037
1             0.718506            0.233887            0.220703
2             0.718506            0.233887            0.220703
3             0.960205            0.190430            0.051270
4             0.960205            0.190430            0.051270
5             0.935059            0.127197           -0.030029
6             0.935059            0.127197           -0.030029
7             0.911133            0.191895           -0.025879
8             0.963867            0.259033            0.033447
9             0.968018            0.287842            0.178955
10            0.953369            0.293701            0.094482
11            0.968262            0.444580            0.054443
12            0.974365            0.365479            0.104980
13            0.937256            0.462891            0.149414
14            0.880615            0.410156            